<a href="https://colab.research.google.com/github/mindyng/Covid19/blob/master/Paper_Study_Design_Annotation_BaselineModel_TreeModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of this Analysis is to get baseline performance using traditional models for Multi-Label Text Classification.  (Deadline: 04.16.20)

## Text = Pre-processed text from: 

NLPDatasets_v7_preprocessed_v7_dataset.csv

Need to combine publication text into one row so there is 1:1 relationship between X and y.

## Labels = Different Annotations with Clinical significance

Need to make sure 1:1  relationship between X and y.
Get rid of unlabeled publications from Study_Design_Annotation_Snapshot_04_12-20 and drop General Notes column. Join remaining rows on cord_uid with pre-processed text file.


In [0]:
import pandas as pd
import numpy as np
import spacy
import gensim
import nltk;  nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
nlp = spacy.load('en', disable=['parser', 'ner'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [0]:
NLPData=pd.read_csv('NLPDatasets_v7_preprocessed_v7_dataset.csv')
NLPData

,paper_id,section,text
0,08a40e31fde744729050ab037f80aa67c7d1053f,title,Global versus focused isolation during the SAR...
1,08a40e31fde744729050ab037f80aa67c7d1053f,abstract,Background: The novel coronavirus (SARS-CoV-2)...
2,08a40e31fde744729050ab037f80aa67c7d1053f,introduction,Since its identification at the very end of 20...
3,08a40e31fde744729050ab037f80aa67c7d1053f,introduction,"(1). Following China, many countries around th..."
4,08a40e31fde744729050ab037f80aa67c7d1053f,introduction,"However, a few countries, such as South Korea,..."
...,...,...,...
9144,c209c6c9d7c022e0c4b997e2d942c0928eac537c,title,-NC-ND 4.0 license
9145,c209c6c9d7c022e0c4b997e2d942c0928eac537c,abstract,[]
9146,c209c6c9d7c022e0c4b997e2d942c0928eac537c,NaN,"epidemic, African migrants residing in Wuhan, ..."
9147,c209c6c9d7c022e0c4b997e2d942c0928eac537c,NaN,African community organisations also compile a...


1. Group by ID to get pub as one row with corresponding text.
2. Pre-process text

In [0]:
NLPData2 = NLPData.drop(['section'], axis=1)
NLPData2['text'] = NLPData2['text'].fillna('').astype(str)
NLPData2 = NLPData2.groupby('paper_id')['text'].apply(' '.join).reset_index()
NLPData2

,paper_id,text
0,01ceb817f8665ed0047c4675e04b67b199e18832,Journal Pre-proof The Chinese herbal prescript...
1,023aa7bdf9b73b7453051e8fd33f9e80df951001,"[] That same day, the Coronavirus Study Group..."
2,0329e0f2e3b8ab0eabe2afe0e5214cc88c61080e,"Journal Pre-proof Favipiravir, an anti-influen..."
3,038d47e5e2526b54dd06cb90340d99b0252b90e7,Association of radiologic findings with mortal...
4,04fae0c0195e39e5bcfb0ddb7a56d4e22944e28f,Journal Pre-proof Novel decoy cellular vaccine...
...,...,...
311,fd14ed7c073b7ff03afa517e9c0fd1e849878252,Direct RNA sequencing and early evolution of S...
312,fe297e7210ed7586d1d84f774ca15a60b5e70173,Coronavirus (COVID-19) Outbreak: What the Depa...
313,fede3c3576d09710757e066a2315889044118c55,Integrated infection control strategy to minim...
314,ff9b8ea0d7aeaf02cbe9943601c0d00ae0aefd93,Investigation of three clusters of COVID-19 in...


In [0]:
NLPData2["text"][0]

"Journal Pre-proof The Chinese herbal prescription JZ-1 induces autophagy to protect against herpes simplex Virus-2 in human vaginal epithelial cells by inhibiting the PI3K/Akt/mTOR pathway The Chinese herbal prescription JZ-1 induces autophagy to protect against herpes simplex Virus-2 in human vaginal epithelial cells by inhibiting the PI3K/Akt/mTOR pathway, Journal of Ethnopharmacology (2020), doi: https://doi. Genital herpes is one of the most common, persistent and highly contagious The results were analyzed by the 2 −∆∆CT method with β-actin as an internal control. The sequences of the primers used in this study are listed in Table 2 . anti-p-mTOR, anti-mTOR, anti-gD, anti-ICP5 and anti-β-actin) at 4°C. After washing 267 three times with TBST for 10 min each, the membranes were incubated with 268 secondary antibodies for 1 h at room temperature, washed another three times, and 269 then visualized by an Odyssey Infrared Imaging system (LI-COR Biosciences, USA). The band densities w

In [0]:

NLPData2.text.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
#NLPData2['text'] = df2['text'].map(lambda x: re.sub('[,\.!?]', '', x))
NLPData2['text']=NLPData2['text'].map(lambda x: x.lower())
NLPData2['text']=NLPData2['text'].str.strip()
NLPData2['text'][0]  

"journal pre-proof the chinese herbal prescription jz-1 induces autophagy to protect against herpes simplex virus-2 in human vaginal epithelial cells by inhibiting the pi3k/akt/mtor pathway the chinese herbal prescription jz-1 induces autophagy to protect against herpes simplex virus-2 in human vaginal epithelial cells by inhibiting the pi3k/akt/mtor pathway, journal of ethnopharmacology (2020), doi: https://doi. genital herpes is one of the most common, persistent and highly contagious the results were analyzed by the 2 ct method with -actin as an internal control. the sequences of the primers used in this study are listed in table 2 . anti-p-mtor, anti-mtor, anti-gd, anti-icp5 and anti--actin) at 4c. after washing 267 three times with tbst for 10 min each, the membranes were incubated with 268 secondary antibodies for 1 h at room temperature, washed another three times, and 269 then visualized by an odyssey infrared imaging system (li-cor biosciences, usa). the band densities were qu

In [0]:
#this pulls out numbers tho :(
NLPData2['text'] = NLPData2.text.map(lambda x: list(gensim.utils.simple_preprocess(str(x), deacc=True)))



In [0]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
NLPData2['text']=NLPData2.text.map(lambda x: lemmatization(x))

In [0]:
NLPData2['text'][0]

[['o', 'r', 'n', 'l'],
 ['p', 'r', 'e'],
 ['r', 'o', 'o'],
 ['t', 'h', 'e'],
 ['h', 'i', 'n', 'e', 's', 'e'],
 ['e', 'r', 'b', 'l'],
 ['r', 'e', 's', 'c', 'r', 'i', 'p', 't', 'o', 'n'],
 [],
 ['c', 'e', 's'],
 ['t', 'o', 'p', 'h', 'a', 'g', 'y'],
 [],
 ['r', 'o', 't', 'e', 'c', 't'],
 ['g', 'a', 'i', 'n', 's', 't'],
 ['e', 'r', 'p', 'e', 's'],
 ['p', 'l', 'e'],
 ['i', 'r', 'u', 's'],
 [],
 ['m', 'a', 'n'],
 ['g', 'n', 'l'],
 ['p', 'i', 't', 'h', 'e', 'l', 'l'],
 ['c', 'l'],
 [],
 ['b', 'i', 't', 'i', 'n', 'g'],
 ['t', 'h', 'e'],
 [],
 ['k', 't'],
 ['t', 'o', 'r'],
 ['a', 't', 'h', 'w', 'a', 'y'],
 ['t', 'h', 'e'],
 ['h', 'i', 'n', 'e', 's', 'e'],
 ['e', 'r', 'b', 'l'],
 ['r', 'e', 's', 'c', 'r', 'i', 'p', 't', 'o', 'n'],
 [],
 ['c', 'e', 's'],
 ['t', 'o', 'p', 'h', 'a', 'g', 'y'],
 [],
 ['r', 'o', 't', 'e', 'c', 't'],
 ['g', 'a', 'i', 'n', 's', 't'],
 ['e', 'r', 'p', 'e', 's'],
 ['p', 'l', 'e'],
 ['i', 'r', 'u', 's'],
 [],
 ['m', 'a', 'n'],
 ['g', 'n', 'l'],
 ['p', 'i', 't', 'h', 'e', 

In [0]:
from google.colab import files
files.upload()

Saving Study_Design_Annotation_Snapshot_4_12_20.csv to Study_Design_Annotation_Snapshot_4_12_20.csv


{'Study_Design_Annotation_Snapshot_4_12_20.csv': b'Assignee,cord_uid,sha,title,url,Computational ,Non-human studies,,Clinical studies,,,,,,,General Notes\r\n,,,,,in silico,in vitro,in vivo,Systematic reivew or meta-analysis of RCTs,RCT,Non-randomized controlled trial,"Comparative study with a concurrent or historical comparator (cohort studies, case-control studies, 2 or more single-arm studies, or interrupted time series with a control group)","Descriptive study without a comparator (single-arm pre-post studies, case series, case reports)",Systematic review or meta-analysis of studies other than RCTs,"Other (please describe, e.g., commentary, editorial, non-systematic review, expert opinion)",Use this space to jot down special notes about an article\r\nZayd,mmykzj9w,f0dd0852e4b1b5ccb3add01fc3aa561478b8be89,Biological effects of HBV X protein on hepatocellular carcinogenesis in association with cellular factors,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7090866/,,,,,,,,,,Non-systemat

In [0]:
df2 = pd.read_csv('/content/Study_Design_Annotation_Snapshot_4_12_20.csv')
df2

,Assignee,cord_uid,sha,title,url,Computational,Non-human studies,Unnamed: 7,Clinical studies,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,General Notes
0,NaN,NaN,NaN,NaN,NaN,in silico,in vitro,in vivo,Systematic reivew or meta-analysis of RCTs,RCT,Non-randomized controlled trial,Comparative study with a concurrent or histori...,Descriptive study without a comparator (single...,Systematic review or meta-analysis of studies ...,"Other (please describe, e.g., commentary, edit...",Use this space to jot down special notes about...
1,Zayd,mmykzj9w,f0dd0852e4b1b5ccb3add01fc3aa561478b8be89,Biological effects of HBV X protein on hepatoc...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of previous studies incl...,NaN
2,Zayd,480bzpi1,a484db449f6d41927b645a3d5bcdd1ada691e694,Do we have enough evidence how seasonal influe...,https://doi.org/10.1016/j.vaccine.2016.04.096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
3,Zayd,n5o7u56f,862dbb7e5c850f946575d3c882f46be61d139a22,Bioactive principles from Cordyceps sinensis: ...,https://doi.org/10.1016/j.jff.2013.04.018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of literature surroundin...,NaN
4,Zayd,8rl4kswe,8acb82e7915763480c44bce48a261d1a3199cda0,Virus Entry: Open Sesame,https://doi.org/10.1016/j.cell.2006.02.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of literature surroundin...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,NaN,dxl7wvcx,f14e7b5fff3c2ac074e61beacf4fb1bba524e438,Anti-Inflammatory and Anti-Allergic Activities...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,ldfa616a,0660d9281c1607ca1ca9cb62dfa00b56f49ddee1,The Last Ten Years of Advancements in Plant-De...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,8y3iuqn4,0f5888442a92665b65f719dac5712fd287f842b8,Gaseous pollutant transmission through windows...,https://doi.org/10.1016/j.enbuild.2017.08.025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999,NaN,ubp6hhpo,370d59bf7f643eb4a7f419b7cf31b3b67c6e0b13,Biological characteristics and propagation of ...,https://doi.org/10.1016/j.virol.2012.11.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df2 = df2.iloc[1:]

In [0]:
df2 = df2.dropna(how='all',subset=['Computational ','Non-human studies','Unnamed: 7','Clinical studies','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14', 'General Notes'])
df2

,Assignee,cord_uid,sha,title,url,Computational,Non-human studies,Unnamed: 7,Clinical studies,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,General Notes
1,Zayd,mmykzj9w,f0dd0852e4b1b5ccb3add01fc3aa561478b8be89,Biological effects of HBV X protein on hepatoc...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of previous studies incl...,NaN
2,Zayd,480bzpi1,a484db449f6d41927b645a3d5bcdd1ada691e694,Do we have enough evidence how seasonal influe...,https://doi.org/10.1016/j.vaccine.2016.04.096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
3,Zayd,n5o7u56f,862dbb7e5c850f946575d3c882f46be61d139a22,Bioactive principles from Cordyceps sinensis: ...,https://doi.org/10.1016/j.jff.2013.04.018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of literature surroundin...,NaN
4,Zayd,8rl4kswe,8acb82e7915763480c44bce48a261d1a3199cda0,Virus Entry: Open Sesame,https://doi.org/10.1016/j.cell.2006.02.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of literature surroundin...,NaN
5,Zayd,whfyczcj,f6fdeab5faa3141b4d0ba262210e6662a7aa261a,Hijacking of the Ubiquitin/Proteasome Pathway ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of particular proteosome...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,NaN,01b0vnnm,2a7c951e191425fd9fa5ac108f07a1f02eb75872,The changing phenotype of microglia from homeo...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,editorial,NaN
644,NaN,hrs7e1ys,6ac46cebdca71f2e25d8a884c5d3ce11fcaf16e7,Efficacy and safety of iota-carrageenan nasal ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
645,NaN,ya5kscsl,f65cdf4a48c1ad11e613449dad480736e9d26945,Snapshot of Viral Infections in Wild Carnivore...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,NaN,omui3nw3,36ad3ea5e6380e9b6c4be753556f94d7355921fb,Chapter 3 Biology and Diseases of Mice,https://doi.org/10.1016/b978-012263951-7/50006-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,book chapter,NaN


In [0]:
df2 = df2.drop(['General Notes'], axis=1)
df2

,Assignee,cord_uid,sha,title,url,Computational,Non-human studies,Unnamed: 7,Clinical studies,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
1,Zayd,mmykzj9w,f0dd0852e4b1b5ccb3add01fc3aa561478b8be89,Biological effects of HBV X protein on hepatoc...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of previous studies incl...
2,Zayd,480bzpi1,a484db449f6d41927b645a3d5bcdd1ada691e694,Do we have enough evidence how seasonal influe...,https://doi.org/10.1016/j.vaccine.2016.04.096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3,Zayd,n5o7u56f,862dbb7e5c850f946575d3c882f46be61d139a22,Bioactive principles from Cordyceps sinensis: ...,https://doi.org/10.1016/j.jff.2013.04.018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of literature surroundin...
4,Zayd,8rl4kswe,8acb82e7915763480c44bce48a261d1a3199cda0,Virus Entry: Open Sesame,https://doi.org/10.1016/j.cell.2006.02.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of literature surroundin...
5,Zayd,whfyczcj,f6fdeab5faa3141b4d0ba262210e6662a7aa261a,Hijacking of the Ubiquitin/Proteasome Pathway ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-systematic review of particular proteosome...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,NaN,01b0vnnm,2a7c951e191425fd9fa5ac108f07a1f02eb75872,The changing phenotype of microglia from homeo...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,editorial
644,NaN,hrs7e1ys,6ac46cebdca71f2e25d8a884c5d3ce11fcaf16e7,Efficacy and safety of iota-carrageenan nasal ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
645,NaN,ya5kscsl,f65cdf4a48c1ad11e613449dad480736e9d26945,Snapshot of Viral Infections in Wild Carnivore...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
646,NaN,omui3nw3,36ad3ea5e6380e9b6c4be753556f94d7355921fb,Chapter 3 Biology and Diseases of Mice,https://doi.org/10.1016/b978-012263951-7/50006-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,book chapter


# Now , need to do inner join between .

In [0]:
df2.merge(NLPData,left_on='sha',right_on='paper_id')

,Assignee,cord_uid,sha,title,url,Computational,Non-human studies,Unnamed: 7,Clinical studies,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,paper_id,section,text
0,NaN,6qctb9bk,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,Process Characterization and Biophysical Analy...,https://doi.org/10.1016/j.xphs.2020.02.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,process characterization study on a new fermen...,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,title,Journal Pre-proof Process Characterization and...
1,NaN,6qctb9bk,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,Process Characterization and Biophysical Analy...,https://doi.org/10.1016/j.xphs.2020.02.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,process characterization study on a new fermen...,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,abstract,[]
2,NaN,6qctb9bk,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,Process Characterization and Biophysical Analy...,https://doi.org/10.1016/j.xphs.2020.02.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,process characterization study on a new fermen...,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,introduction,Leishmaniasis is a neglected tropical disease ...
3,NaN,6qctb9bk,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,Process Characterization and Biophysical Analy...,https://doi.org/10.1016/j.xphs.2020.02.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,process characterization study on a new fermen...,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,introduction,Cutaneous leishmaniasis (CL) is the most commo...
4,NaN,6qctb9bk,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,Process Characterization and Biophysical Analy...,https://doi.org/10.1016/j.xphs.2020.02.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,process characterization study on a new fermen...,e61c31e3d33e5a165b698a1a89a8ca7781d7e805,introduction,"Afghanistan, Iran, Syria, Algeria, Brazil, and..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,NaN,rlpzejjt,148b47504613ad536de7ad3c642157c4a70d9d9e,The spike glycoprotein of the new coronavirus ...,https://doi.org/10.1016/j.antiviral.2020.104742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,148b47504613ad536de7ad3c642157c4a70d9d9e,NaN,"Since furin is highly expressed in lungs, an e..."
77,NaN,rlpzejjt,148b47504613ad536de7ad3c642157c4a70d9d9e,The spike glycoprotein of the new coronavirus ...,https://doi.org/10.1016/j.antiviral.2020.104742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,148b47504613ad536de7ad3c642157c4a70d9d9e,NaN,Obviously much more work is needed to demonstr...
78,NaN,rlpzejjt,148b47504613ad536de7ad3c642157c4a70d9d9e,The spike glycoprotein of the new coronavirus ...,https://doi.org/10.1016/j.antiviral.2020.104742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,148b47504613ad536de7ad3c642157c4a70d9d9e,NaN,A variety of approaches have been proposed to ...
79,NaN,rlpzejjt,148b47504613ad536de7ad3c642157c4a70d9d9e,The spike glycoprotein of the new coronavirus ...,https://doi.org/10.1016/j.antiviral.2020.104742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,148b47504613ad536de7ad3c642157c4a70d9d9e,figref0,Characterization .2). The phylogenetic tree wa...


In [0]:

df2.text.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
#df2['text'] = df2['text'].map(lambda x: re.sub('[,\.!?]', '', x))
df2['text'].map(lambda x: x.lower())
df2['text'].str.strip()
stop_words = stopwords.words('english')


In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
